In [1]:
class Robot:
    def __init__(self, board):
        self.board = board
        self.coordinates = self.get_init_coordinates()

    def print_board(self):
        for row in self.board:
            print(row)
        print()

    def get_init_coordinates(self):
        for i, row in enumerate(self.board):
            if '@' in row:
                return i, row.index('@')

    def get_objects_in_front(self, instruction):
        x, y = self.coordinates
        objects_in_front = []
        if instruction in ['^', 'v']:
            if instruction == 'v':
                range_ = range(x+1, len(self.board))
            else:
                range_ = range(x-1, -1, -1)
            y_to_search = {y}
            for i in range_:
                new_y_to_search = set(y_to_search)
                for j in y_to_search:
                    if self.board[i][j] == '[':
                        new_y_to_search.add(j+1)
                    elif self.board[i][j] == ']':
                        new_y_to_search.add(j-1)
                    elif self.board[i][j] == '.':
                        new_y_to_search.remove(j)
                    elif self.board[i][j] == '#':
                        objects_in_front.append((self.board[i][j], (i, j)))
                        return objects_in_front
                y_to_search = new_y_to_search
                for j in y_to_search:
                    objects_in_front.append((self.board[i][j], (i, j)))
        if instruction == '>':
            for i in range(y+1, len(self.board[0])):
                objects_in_front.append((self.board[x][i], (x, i)))
        if instruction == '<':
            for i in range(y-1, -1, -1):
                objects_in_front.append((self.board[x][i], (x, i)))
        return objects_in_front

    def is_move_valid(self, instruction, objects_in_front):
        if instruction in ['<', '>']:
            str_objects = ''.join([i[0] for i in objects_in_front])
            if '.' in str_objects.split('#')[0]:
                return True
        elif instruction in ['^', 'v']:
            for object_, _ in objects_in_front:
                if object_ == '#':
                    return False
            return True
        return False

    def update_board(self, old_coordinates, new_coordinates):
        x0, y0 = old_coordinates
        x1, y1 = new_coordinates
        self.board[x1] = self.board[x1][:y1] + self.board[x0][y0] + self.board[x1][y1+1:]
        self.board[x0] = self.board[x0][:y0] + '.' + self.board[x0][y0+1:]

    def move(self, coordinates, instruction):
        if instruction == '^':
            new_coordinates = coordinates[0] - 1, coordinates[1]
        elif instruction == '>':
            new_coordinates = coordinates[0], coordinates[1] + 1
        elif instruction == 'v':
            new_coordinates = coordinates[0] + 1, coordinates[1]
        elif instruction == '<':
            new_coordinates = coordinates[0], coordinates[1] - 1
        self.update_board(coordinates, new_coordinates)
        return new_coordinates

    def get_object_coordinates_to_move(self, instruction, objects_in_front):
        object_coordinates_to_move = []
        for object_, coordinates in objects_in_front:
            if object_ in ['O', '[', ']']:
                object_coordinates_to_move.append(coordinates)
            else:
                break
        return object_coordinates_to_move[::-1]

    def follow_instructions(self, instructions):
        self.score = 0
        for instruction in instructions:
            self.objects_in_front = self.get_objects_in_front(instruction)
            if self.is_move_valid(instruction, self.objects_in_front):
                objects_to_move = self.get_object_coordinates_to_move(instruction, self.objects_in_front)
                for coordinates in objects_to_move:
                    self.move(coordinates, instruction)
                self.coordinates = self.move(self.coordinates, instruction)
        for x in range(len(self.board)):
            for y in range(len(self.board[0])):
                if self.board[x][y] in ['O', '[']:
                    self.score += 100*x + y
        return self.score

    def extend_board(self):
        extended_board = []
        for x in range(len(self.board)):
            extended_row = ''
            for y in range(len(self.board[0])):
                tile = self.board[x][y]
                if tile == '#':
                    extended_row += '##'
                elif tile == 'O':
                    extended_row += '[]'
                elif tile == '.':
                    extended_row += '..'
                elif tile == '@':
                    extended_row += '@.'
            extended_board.append(extended_row)
        self.board = extended_board
        self.coordinates = self.get_init_coordinates()


def load_data(path):
    with open(path) as f:
        data = f.read().split('\n\n')
    return data


def process_data(data):
    board = data[0].split()
    instructions = data[1].replace('\n', '')
    return board, instructions


data = load_data('input.txt')
board, instructions = process_data(data)

robot1 = Robot(board)
print(robot1.follow_instructions(instructions))

robot2 = Robot(board)
robot2.extend_board()
print(robot2.follow_instructions(instructions))

1371036
1390010
